## 0. 简介

本 Note 记录使用 Neural Networks (以下简称NN）做自然语言处理时所用的工具函数，包括：

1. `naive_preprocess(text)` 将输入文本字符串转换为单词 ID 的 Numpy 数组，并返回对应的单词-ID 字典和 ID-单词字典

2. `create_co_matrix(corpus, vocab_size, window_size=1)` 根据词表和窗口大小，在以单词 ID 表现的语料库上构建共现矩阵

3. `ppmi(C, verbose=False, eps= 1e-8)` 基于共现矩阵计算 PPMI (positive point-wise mutual information) 共现矩阵

4. `cos_similarity(x, y, eps=1e-8)` 计算两个向量之间的余弦相似度 

5. `most_similay(query, word_to_id, id_to_word, word_matrix, top=5)` 找出 corpus 中和 query 最相近的 top 个单词

## 1. naive_preprocess()

输入参数为文本序列，输出为该文本的单词 ID 序列，word_to_id 字典以及 id_to_word 字典

In [1]:
def naive_preprocess(text):
    '''基于输入文本构建语料库，附带构建单词-ID 和 ID-单词两份字典，仅处理以 . 结尾的句子文本
    
    :param text: 待处理的文本字符串
    
    :return corpus: 单词 ID 组成的 Numpy 数组
    :return word_to_id: key 是单词，value 是对应的 ID 的字典
    :return id_to_word: key 是 ID，value 是对应的单词的字典
    '''
    
    text = text.lower()
    text = text.replace('.', ' .') # 在文本末尾的句号前添加空格
    words = text.split(' ') # 以空格为定界符将输入文本拆分为单词
    
    word_to_id = {}
    id_to_word = {}
    
    for word in words:
        if word not in word_to_id:
            new_id = len(word_to_id)
            word_to_id[word] = new_id  # 核心就是构建 word_to-id 字典
            id_to_word[new_id] = word
    
    corpus = np.array([word_to_id[w] for w in words])  # 依 word_to_id 构建 corpus
        
    return corpus, word_to_id, id_to_word

## 2 create_co_matrix()

根据给定的词表和窗口，在语料库上统计构建贡献矩阵

主要参数包括：

1. `corpus`，提供统计数据的语料库，其中值是单词 ID
2. `vocab_size` 词汇表大小，共现矩阵是一个 (vocab_size, vocab_size) 的方阵
3. `window_size` 目标单词左右两边算作上下文的单词数目

返回的共现矩阵的行和列都是用 单词 ID 标定的，这进一步说明构建语料库时的 `word_to_id` 和 `id_to_word` 两个字典的重要性

In [1]:
def create_co_matrix(corpus, vocab_size, window_size=1):
    '''基于 corpus 构建共现矩阵
    
    Paramters
    ---------
    corpus : 单词 ID 形式的语料库
    vocab_size : 词汇表大小
    window_size : 上下文窗口大小，这里给出的是单边窗口大小，实际窗口要乘以 2
    
    Return
    ------
    基于次数的共现矩阵
    '''
    
    corpus_size = len(corpus_size)
    # 初始化共现矩阵
    co_matrix = np.zeros((vocab_size, vocab_size), dtype=np.int32)
    
    ## 关键步骤
    ## 遍历 corpus 中每个单词
    for idx, word_id in enumerate(corpus):
        for i in range(1, window_size + 1):  # 依次统计目标单词的上下文单词，按单边窗口计算单词下标范围
            left_idx = idx - i
            right_idx = idx + i
            
            if left_idx >= 0:
                left_word_id = corpus[left_idx]
                co_matrix[word_id, left_word_id] += 1  # 注意这里是加法操作，因为是统计累计值
            
            if right_idx < corpus_size:
                right_word_id = corpus[right_idx]
                co_matrix[word_id, right_word_id] += 1
                
    return co_matrix

## 3. cos_similarity()

输入参数为两个向量，返回两个向量的余弦相似度

In [2]:
def cos_similarity(x, y, eps=1e-8):
    '''计算两个向量 x 和 y 的余弦相似度
    
    :param x: 向量 x
    :param y: 向量 y
    :param eps: 避免出现零向量的 L2 范数为 0 的情况
    
    :return: 返回 x 和 y 的余弦相似度
    '''
    
    ## x 和 y 执行正规化，除以各自的 L2 范数
    nx = x / (np.sqrt(np.sum(x ** 2)) + eps)  # eps 防止零向量导致的分母为 0 的情况
    ny = y / (np.sqrt(np.sum(y ** 2)) + eps)  # eps=1e-8 一般会在浮点数近似表示时被“吸收“”
    
    return np.dot(nx, ny)

## 4. most_similar()

